In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Gaucher_Disease"
cohort = "GSE124283"

# Input paths
in_trait_dir = "../../input/GEO/Gaucher_Disease"
in_cohort_dir = "../../input/GEO/Gaucher_Disease/GSE124283"

# Output paths
out_data_file = "../../output/preprocess/Gaucher_Disease/GSE124283.csv"
out_gene_data_file = "../../output/preprocess/Gaucher_Disease/gene_data/GSE124283.csv"
out_clinical_data_file = "../../output/preprocess/Gaucher_Disease/clinical_data/GSE124283.csv"
json_path = "../../output/preprocess/Gaucher_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Changes in the level of expression of genes involved in the pathogenic mechanisms in rare, inherited metabolic diseases."
!Series_summary	"Inherited metabolic diseases belong to the group of rare diseases (so called ‘orphan diseases’) whose incidence is less than 1: 5 000 live births. Among these diseases the lysosomal storage diseases (LSD) are also distinguished, which are caused by disorders in the lysosomal system resulting from the mutations in the genes coding for lysosomal hydrolases, cofactors, enzymes involved in the posttranslational processing, and proteins present in the lysosomal membrane.  Although about 70 LSD are recognized so far, their pathomechanism is almost unknown. Hitherto existing results of scientific investigations indicate that different cellular pathways and events are involved in the pathogenic processes: autophagy, apoptosis, toxic action of lyso- derivatives of lipid compounds, disordered  Ca2+ ions intracellular hom

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Optional, Callable

# Check if this cohort contains gene expression data
# Based on the background information, this dataset uses microarray analysis with Illumina technology,
# which suggests it contains gene expression data
is_gene_available = True

# Analyze the sample characteristics dictionary
# trait_row: The 'condition' field at index 2 contains information about Gaucher Disease (our trait)
trait_row = 2

# Check age_row: There's no clear age information in the sample characteristics
age_row = None

# Check gender_row: There's gender information at index 3
gender_row = 3

# Define conversion functions
def convert_trait(value):
    """Convert trait information to binary format (1 for Gaucher Disease, 0 for control)."""
    if value is None or ':' not in value:
        return None
    
    condition = value.split(':', 1)[1].strip()
    
    # 1 for Gaucher Disease, 0 for control
    if 'Gaucher' in condition:
        return 1
    elif 'Control' in condition:
        return 0
    else:
        # All other conditions (NPC variants) are not relevant for our Gaucher_Disease study
        return None

def convert_age(value):
    """Convert age to continuous format."""
    # No age data available
    return None

def convert_gender(value):
    """Convert gender to binary format (0 for female, 1 for male)."""
    if value is None or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip()
    
    # In the data 'M' appears to be Male and 'K' appears to be Female (likely from a non-English language)
    if gender == 'M':
        return 1
    elif gender == 'K':
        return 0
    else:
        return None

# Check if trait data is available
is_trait_available = trait_row is not None

# Validate and save initial cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If trait data is available, extract clinical features
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in the previous output
    # The dictionary contains lists of values for each characteristic
    sample_chars_dict = {0: ['patient: K12', 'patient: K14', 'patient: SB', 'patient: SK', 'patient: 91/78', 'patient: K19', 'patient: DA', 'patient: SP', 'patient: K6', 'patient: K7', 'patient: WP', 'patient: BZ', 'patient: K4', 'patient: K5', 'patient: BE', 'patient: KM', 'patient: K8', 'patient: K9', 'patient: K13', 'patient: K15', 'patient: ML', 'patient: GP', 'patient: 14/84', 'patient: 69/84', 'patient: ZJ', 'patient: BB', 'patient: K10', 'patient: K11', 'patient: NP', 'patient: SK_1'], 
                          1: ['viability: alive', 'viability: deceased', 'viability: nie alive', 'viability: N/A'], 
                          2: ['condition: Control', 'condition: NPC-D', 'condition: NPC-N', 'condition: NPC-M', 'condition: NPC?', 'condition: NPC(-)variant?', 'condition: NPC-NBP', 'condition: Gaucher t.1', 'condition: Gaucher', 'condition: NPC(-)-variant', 'condition: N/A', 'condition: NPC(-)'], 
                          3: ['gender: M', 'gender: K', 'gender: N/A']}
    
    # Convert the dictionary to a DataFrame
    # We need to transpose the data so that each row represents a characteristic
    clinical_data = pd.DataFrame.from_dict(sample_chars_dict, orient='index')
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV file
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, 1.0], 1: [nan, 0.0], 2: [nan, nan], 3: [nan, nan], 4: [nan, nan], 5: [nan, nan], 6: [nan, nan], 7: [1.0, nan], 8: [1.0, nan], 9: [nan, nan], 10: [nan, nan], 11: [nan, nan], 12: [nan, nan], 13: [nan, nan], 14: [nan, nan], 15: [nan, nan], 16: [nan, nan], 17: [nan, nan], 18: [nan, nan], 19: [nan, nan], 20: [nan, nan], 21: [nan, nan], 22: [nan, nan], 23: [nan, nan], 24: [nan, nan], 25: [nan, nan], 26: [nan, nan], 27: [nan, nan], 28: [nan, nan], 29: [nan, nan]}
Clinical data saved to ../../output/preprocess/Gaucher_Disease/clinical_data/GSE124283.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 65
Header line: "ID_REF"	"GSM3526881"	"GSM3526882"	"GSM3526883"	"GSM3526884"	"GSM3526885"	"GSM3526886"	"GSM3526887"	"GSM3526888"	"GSM3526889"	"GSM3526890"	"GSM3526891"	"GSM3526892"	"GSM3526893"	"GSM3526894"	"GSM3526895"	"GSM3526896"	"GSM3526897"	"GSM3526898"	"GSM3526899"	"GSM3526900"	"GSM3526901"	"GSM3526902"	"GSM3526903"	"GSM3526904"	"GSM3526905"	"GSM3526906"	"GSM3526907"	"GSM3526908"	"GSM3526909"	"GSM3526910"	"GSM3526911"	"GSM3526912"	"GSM3526913"	"GSM3526914"	"GSM3526915"	"GSM3526916"	"GSM3526917"	"GSM3526918"	"GSM3526919"	"GSM3526920"	"GSM3526921"	"GSM3526922"	"GSM3526923"	"GSM3526924"	"GSM3526925"	"GSM3526926"	"GSM3526927"	"GSM3526928"	"GSM3526929"	"GSM3526930"	"GSM3526931"	"GSM3526932"	"GSM3526933"	"GSM3526934"	"GSM3526935"	"GSM3526936"	"GSM3526937"	"GSM3526938"	"GSM3526939"	"GSM3526940"	"GSM3526941"	"GSM3526942"	"GSM3526943"	"GSM3526944"	"GSM3526945"	"GSM3526946"	"GSM3526947"	"GSM3526948"	"GSM3526949"	"GSM3526950"	"GSM3526951"	"GSM3526952"	"GSM3526953"	

Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Let's review the gene identifiers in the expression data
# Looking at the first few identifiers:
# '7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', etc.

# These appear to be standard human gene symbols. For example:
# - A1BG: Alpha-1-B Glycoprotein
# - A2M: Alpha-2-Macroglobulin
# - AAAS: Achalasia, Adrenocortical Insufficiency, Alacrimia Syndrome

# Since these are already in the standard HGNC gene symbol format, 
# no mapping is required.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Extract the gene expression data and normalize gene symbols
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)
print(f"Original gene data shape: {gene_data.shape}")

# Normalize gene symbols using NCBI Gene database
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data created in step 2
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")

# Check clinical data structure and rename columns if needed
print("Clinical data columns:", clinical_df.columns.tolist())
print("Clinical data head:", clinical_df.head())

# Ensure clinical data has appropriate column names
if trait not in clinical_df.columns and len(clinical_df.columns) >= 1:
    # Assuming the first column is the trait column based on the convert_trait function
    clinical_df = clinical_df.rename(columns={clinical_df.columns[0]: trait})
    if len(clinical_df.columns) >= 2:
        # Assuming the second column might be Gender based on the convert_gender function
        clinical_df = clinical_df.rename(columns={clinical_df.columns[1]: 'Gender'})

print("Clinical data after renaming columns:", clinical_df.columns.tolist())

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data columns:", linked_data.columns[:10].tolist())
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# Verify the trait column exists in linked data
if trait not in linked_data.columns:
    print(f"WARNING: '{trait}' column not found in linked data")
    # Try to identify which column might contain the trait data
    for col in linked_data.columns:
        if linked_data[col].nunique() <= 2 and col not in ['Gender', 'Age']:
            print(f"Column {col} has {linked_data[col].nunique()} unique values and might be the trait column")
            linked_data = linked_data.rename(columns={col: trait})
            break

# 3. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
if 'Age' in linked_data.columns:
    print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
if 'Gender' in linked_data.columns:
    print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")

gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 4. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 5. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis."
)

# 6. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Original gene data shape: (31424, 144)
Gene data shape after normalization: (20747, 144)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Normalized gene data saved to ../../output/preprocess/Gaucher_Disease/gene_data/GSE124283.csv
Loaded clinical data shape: (2, 30)
Clinical data columns: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
Clinical data head:      0    1   2   3   4   5   6    7    8   9  ...  20  21  22  23  24  25  \
0  0.0  NaN NaN NaN NaN NaN NaN  1.0  1.0 NaN  ... NaN NaN NaN NaN NaN NaN   
1  1.0  0.0 NaN NaN NaN NaN NaN  NaN  NaN NaN  ... NaN NaN NaN NaN NaN NaN   

   26  27  28  29  
0 NaN NaN NaN NaN  
1 NaN NaN NaN NaN  

[2 rows x 30 columns]
Clinical data after renaming columns: ['Gaucher_Disease', 'Gender', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
Linked data shape: (174, 20749)
Linked data columns: [0, 1, 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GAL